<a href="https://colab.research.google.com/github/AyishaR/Spokendigit/blob/master/Spokendigit_Five_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import tarfile
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import matplotlib
import csv
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

#Data

In [4]:
digit = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

#Dataset 



Spoken_mnist_finalfts.csv is generated in 'Spokenmnist feature extraction.ipynb'.

In [9]:
finalfts = pd.read_csv("Spoken_digit_finalfts.csv")

In [6]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
finalfts[finalfts.columns[1:]] = scale.fit_transform(finalfts[finalfts.columns[1:]])

In [10]:
spokendset = TensorDataset(torch.tensor(np.array(finalfts[finalfts.columns[1:]].astype('float32'))),torch.tensor(finalfts['Label'])) 

In [11]:
# 90-10 split
size = len(spokendset)
val_size = int(0.1 * size)
train_size = size - val_size 

train_dset, val_dset = random_split(spokendset, [train_size, val_size])

train_size, val_size

(21298, 2366)

In [12]:
train_dl = DataLoader(train_dset, 512, True)
val_dl = DataLoader(val_dset, 512)

# Device


In [13]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [14]:
device = get_default_device()
device

device(type='cpu')

In [15]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

#Train

In [16]:
class SpokenDigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(173, 1024)
        self.l2 = nn.Linear(1024, 512)
        self.l3 = nn.Linear(512, 64)
        self.l4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return x

    def training_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        _, pred = torch.max(outputs, 1)
        accuracy = torch.tensor(torch.sum(pred==labels).item()/len(pred))
        return [loss.detach(), accuracy.detach()] 

In [28]:
def evaluate(model, loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in loader]
    outputs = torch.tensor(outputs).T
    loss, accuracy = torch.mean(outputs, dim=1)
    return {"loss" : loss.item(), "accuracy" : accuracy.item()}

In [18]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [19]:
def fit(model, train_loader, val_loader, epochs, lr, optimizer_function = torch.optim.Adam):
    history = []
    optimizer = optimizer_function(model.parameters(), lr)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, epochs=epochs, steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        print("Epoch ", epoch)
        #Train
        model.train()
        lrs = []
        tr_loss = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            tr_loss.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            lrs.append(get_lr(optimizer))
            sched.step()
        #Validate
        result = evaluate(model, val_loader)
        result["lrs"] = lrs
        result["train loss"] = torch.stack(tr_loss).mean().item()
 
        print("Last lr: ", lrs[-1]," Train_loss: ", result["train loss"], " Val_loss: ", result['loss'], " Accuracy: ", result['accuracy'])
        history.append(result)         
    return history

In [32]:
model = to_device(SpokenDigitModel(), device)
history = []
evaluate(model, val_dl)

{'accuracy': 0.10285229980945587, 'loss': 3.1926627159118652}

In [33]:
history.append(fit(model, train_dl, val_dl, 64, 0.01))

Epoch  0



Last lr:  0.00046125331558460707  Train_loss:  2.2663004398345947  Val_loss:  1.9734230041503906  Accuracy:  0.2775648534297943
Epoch  1



Last lr:  0.000649371685235026  Train_loss:  1.7904225587844849  Val_loss:  1.602102518081665  Accuracy:  0.4390698969364166
Epoch  2



Last lr:  0.0009593538973555506  Train_loss:  1.523494839668274  Val_loss:  1.4762526750564575  Accuracy:  0.4833161234855652
Epoch  3



Last lr:  0.0013828987634391283  Train_loss:  1.409789800643921  Val_loss:  1.388493299484253  Accuracy:  0.507468581199646
Epoch  4


...


Last lr:  0.00019545260862094147  Train_loss:  0.0577256977558136  Val_loss:  2.0072267055511475  Accuracy:  0.7595101594924927
Epoch  60



Last lr:  0.00011027567791908655  Train_loss:  0.056645650416612625  Val_loss:  2.016090154647827  Accuracy:  0.7627211213111877
Epoch  61



Last lr:  4.913417694027851e-05  Train_loss:  0.05607904866337776  Val_loss:  2.019317150115967  Accuracy:  0.7623304724693298
Epoch  62



Last lr:  1.232864537599174e-05  Train_loss:  0.05574125424027443  Val_loss:  2.0201499462127686  Accuracy:  0.7615492343902588
Epoch  63



Last lr:  4e-08  Train_loss:  0.055521465837955475  Val_loss:  2.0203850269317627  Accuracy:  0.7619398832321167


In [34]:
@torch.no_grad()
def predict_dl(model, dl):
    torch.cuda.empty_cache()
    batch_probs = []
    batch_targ = []
    for xb, yb in dl:
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
        batch_targ.append(yb.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    batch_targ = torch.cat(batch_targ)
    return [list(values).index(max(values)) for values in batch_probs], batch_targ

In [36]:
r = evaluate(model, val_dl)
yp, yt = predict_dl(model, val_dl)
print("Loss: ", r['loss'], "\nAccuracy: ", r['accuracy'], "\nF-score: ", f1_score(yt, yp, average='micro'))

Loss:  2.0203850269317627 
Accuracy:  0.7619398832321167 
F-score:  0.7586644125105664


In [38]:
torch.save(model, '/content/spokendigit_lr_all.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SpokenDigitModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
